# MNIST training with PyTorch

MNIST is a widely used dataset for handwritten digit classification. It consists of 70,000 labeled 28x28 pixel grayscale images of hand-written digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits). This tutorial will show how to train and test an MNIST model on SageMaker using PyTorch. 



In [12]:
import os
import json

import sagemaker
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role


sess = sagemaker.Session()

role = get_execution_role()

output_path='s3://' + sess.default_bucket() + '/mnist'

## PyTorch Estimator

The `PyTorch` class allows you to run your training script on SageMaker
infrastracture in a containerized environment. In this notebook, we
refer to this container as *training container*. 

You need to configure
it with the following parameters to set up the environment:

- entry_point: A user defined python file to be used by the training container as the 
instructions for training. We further discuss this file in the next subsection.

- role: An IAM role to make AWS service requests

- instance_type: The type of SageMaker instance to run your training script. 
Set it to `local` if you want to run the training job on 
the SageMaker instance you are using to run this notebook

- instance count: The number of instances you need to run your training job. 
Multiple instances are needed for distributed training.

- output_path: 
S3 bucket URI to save training output (model artifacts and output files)

- framework_version: The version of PyTorch you need to use.

- py_version: The python version you need to use

For more information, see [the API reference](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase)



## Implement the entry point for training

The entry point for training is a python script that provides all 
the code for training a PyTorch model. It is used by the SageMaker 
PyTorch Estimator (`PyTorch` class above) as the entry point for running the training job.

Under the hood, SageMaker PyTorch Estimator creates a docker image
with runtime environemnts 
specified by the parameters you used to initiated the
estimator class and it injects the training script into the 
docker image to be used as the entry point to run the container.

In the rest of the notebook, we use *training image* to refer to the 
docker image specified by the PyTorch Estimator and *training container*
to refer to the container that runs the training image. 

This means your training script is very similar to a training script
you might run outside Amazon SageMaker, but it can access the useful environment 
variables provided by the training image. Checkout [the short list of environment variables provided by the SageMaker service](https://sagemaker.readthedocs.io/en/stable/frameworks/mxnet/using_mxnet.html?highlight=entry%20point) to see some common environment 
variables you might used. Checkout [the complete list of environment variables](https://github.com/aws/sagemaker-training-toolkit/blob/master/ENVIRONMENT_VARIABLES.md) for a complete 
description of all environment variables your training script
can access to. 

In this example, we use the training script `code/train.py`
as the entry point for our PyTorch Estimator.


In [13]:
!pygmentize 'code/train.py'

import argparse
import gzip
import json
import logging
import os
import sys

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader


logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))

# Based on https://github.com/pytorch/examples/blob/master/mnist/main.py
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
   

### Set hyperparameters

In addition, PyTorch estimator allows you to parse command line arguments
to your training script via `hyperparameters`.

<span style="color:red"> Note: local mode is not supported in SageMaker Studio </span>

In [14]:
# set local_mode to be True if you want to run the training script
# on the machine that runs this notebook

local_mode=True

if local_mode:
    instance_type='local'
else:
    instance_type='ml.c4.xlarge'
    
est = PyTorch(
    entry_point='train.py',
    source_dir='code', # directory of your training script
    role=role,
    framework_version='1.5.0',
    py_version='py3',
    instance_type=instance_type,
    instance_count=1,
    output_path=output_path,
    hyperparameters={
        'batch-size':128,
        'epochs':20,
        'learning-rate': 1e-3,
        'log-interval':100
    }
)


The training container executes your training script like

```
python train.py --batch-size 100 --epochs 10 --learning-rate 1e-3 \
    --log-interval 100
```

## Set up channels for training and testing data

You need to tell `PyTorch` estimator where to find your training and 
testing data. It can be a link to an S3 bucket or it can be a path
in your local file system if you use local mode. In this example,
we download the MNIST data from a public S3 bucket and upload it 
to your default bucket. 

In [15]:
import logging
import boto3
from botocore.exceptions import ClientError


# Download training and testing data from a public S3 bucket


def download_from_s3(data_dir='/tmp/data', train=True):
    """Download MNIST dataset and convert it to numpy array
    
    Args:
        data_dir (str): directory to save the data
        train (bool): download training set
    
    Returns:
        None
    """
    
    # Get global config
    with open('code/config.json', 'r') as f:
        CONFIG=json.load(f)
    
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    
    if train:
        images_file = "train-images-idx3-ubyte.gz"
        labels_file = "train-labels-idx1-ubyte.gz"
    else:
        images_file = "t10k-images-idx3-ubyte.gz"
        labels_file = "t10k-labels-idx1-ubyte.gz"

    # download objects
    s3 = boto3.client('s3')
    bucket = CONFIG['public_bucket']
    for obj in [images_file, labels_file]:
        key = os.path.join("datasets/image/MNIST", obj)
        dest = os.path.join(data_dir, obj)
        if not os.path.exists(dest):
            s3.download_file(bucket, key, dest)
    return


download_from_s3('/tmp/data', True)
download_from_s3('/tmp/data', False)



In [16]:
# upload to the default bucket

prefix = 'mnist'
bucket = sess.default_bucket()
loc = sess.upload_data(path='/tmp/data', bucket=bucket, key_prefix=prefix)

channels = {
    "training": loc,
    "testing": loc
}


The keys of the dictionary `channels` are parsed to the training image
and it creates the environment variable `SM_CHANNEL_<key name>`. 

In this example, `SM_CHANNEL_TRAINING` and `SM_CHANNEL_TESTING` are created in the training image (checkout 
how `code/train.py` access these variables). For more information,
see: [SM_CHANNEL_{channel_name}](https://github.com/aws/sagemaker-training-toolkit/blob/master/ENVIRONMENT_VARIABLES.md#sm_channel_channel_name)

If you want, you can create a channel for validation:
```
channels = {
    'training': train_data_loc,
    'validation': val_data_loc,
    'test': test_data_loc
    }
```
You can then access this channel within your training script via
`SM_CHANNEL_VALIDATION`


## Run the training script on SageMaker
Now, the training container has everything to execute your training
script. You can start the container by calling `fit` method.

In [17]:
est.fit(inputs=channels)

Using the short-lived AWS credentials found in session. They might expire while running.


Creating tmp0pi1wvq8_algo-1-0lq13_1 ... 
Attaching to tmp0pi1wvq8_algo-1-0lq13_12mdone
algo-1-0lq13_1  | 2020-11-24 06:27:54,974 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
algo-1-0lq13_1  | 2020-11-24 06:27:54,977 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-0lq13_1  | 2020-11-24 06:27:54,988 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
algo-1-0lq13_1  | 2020-11-24 06:27:54,991 sagemaker_pytorch_container.training INFO     Invoking user training script.
algo-1-0lq13_1  | 2020-11-24 06:27:55,140 sagemaker-containers INFO     Module default_user_module_name does not provide a setup.py. 
algo-1-0lq13_1  | Generating setup.py
algo-1-0lq13_1  | 2020-11-24 06:27:55,141 sagemaker-containers INFO     Generating setup.cfg
algo-1-0lq13_1  | 2020-11-24 06:27:55,141 sagemaker-containers INFO     Generating MANIFEST.in
algo-1-0lq13_1  | 2020-11-24 06:27:55,141 sagemaker-conta

algo-1-0lq13_1  | Start training ...
algo-1-0lq13_1  | Train Epoch: 1 [12800/60000 (21%)] Loss: 0.571242
algo-1-0lq13_1  | Train Epoch: 1 [25600/60000 (43%)] Loss: 0.443688
algo-1-0lq13_1  | Train Epoch: 1 [38400/60000 (64%)] Loss: 0.271935
algo-1-0lq13_1  | Train Epoch: 1 [51200/60000 (85%)] Loss: 0.235600
algo-1-0lq13_1  | Test set: Average loss: 0.1149, Accuracy: 9641/10000, 96.41)
algo-1-0lq13_1  | 
algo-1-0lq13_1  | Train Epoch: 2 [12800/60000 (21%)] Loss: 0.248042
algo-1-0lq13_1  | Train Epoch: 2 [25600/60000 (43%)] Loss: 0.201008
algo-1-0lq13_1  | Train Epoch: 2 [38400/60000 (64%)] Loss: 0.272146
algo-1-0lq13_1  | Train Epoch: 2 [51200/60000 (85%)] Loss: 0.171823
algo-1-0lq13_1  | Test set: Average loss: 0.0750, Accuracy: 9773/10000, 97.73)
algo-1-0lq13_1  | 
algo-1-0lq13_1  | Train Epoch: 3 [12800/60000 (21%)] Loss: 0.212504
algo-1-0lq13_1  | Train Epoch: 3 [25600/60000 (43%)] Loss: 0.364843
algo-1-0lq13_1  | Train Epoch: 3 [38400/60000 (64%)] Loss: 0.310983
algo-1-0lq13_1  | T

algo-1-0lq13_1  | Train Epoch: 20 [25600/60000 (43%)] Loss: 0.057095
algo-1-0lq13_1  | Train Epoch: 20 [38400/60000 (64%)] Loss: 0.102891
algo-1-0lq13_1  | Train Epoch: 20 [51200/60000 (85%)] Loss: 0.085810
algo-1-0lq13_1  | Test set: Average loss: 0.0314, Accuracy: 9899/10000, 98.99)
algo-1-0lq13_1  | 
algo-1-0lq13_1  | Saving the model
algo-1-0lq13_1  | 2020-11-24 06:30:00,294 sagemaker-containers INFO     Reporting training SUCCESS
tmp0pi1wvq8_algo-1-0lq13_1 exited with code 0
Aborting on container exit...
===== Job Complete =====


## Inspect and store model data

Now, the training is finished, the model artifact has been saved in 
the `output_path`. We 

In [18]:
pt_mnist_model_data = est.model_data
print("Model artifact saved at:\n", pt_mnist_model_data)

Model artifact saved at:
 s3://sagemaker-us-west-2-688520471316/mnist/pytorch-training-2020-11-24-06-27-47-446/model.tar.gz


We store the variable `model_data` in the current notebook kernel. 
In the [next notebook](get_started_with_mnist_deploy.ipynb), you will learn how to retrieve the model artifact and deploy to a SageMaker
endpoint.

In [ ]:
%store pt_mnist_model_data

## Test and debug the entry point before executing the training container

The entry point `code/train.py` provided here has been tested and it can be executed in the training container. 
When you do develop your own training script, it is a good practice to simulate the container environment 
in the local shell and test it before sending it to SageMaker, because debugging in a containerized environment
is rather cumbersome. The following script shows how you can test your training script:

In [ ]:
!pygmentize code/test_train.py